In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

def load_csv(filename):
    speeds = []
    ammo = []
    labels = []

    with open(filename, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            try:
                s = float(row[0])
                a = float(row[1])
                y = int(row[2])

                speeds.append(s)
                ammo.append(a)
                labels.append(y)
            except:
                continue

    X = np.column_stack((speeds, ammo))
    y = np.array(labels).reshape(-1, 1)
    return X, y


def normalize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_norm = (X - mean) / std
    return X_norm, mean, std

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b):
    m = len(y)
    z = X @ w + b
    h = sigmoid(z)

    cost = -(1/m) * np.sum(
        y * np.log(h + 1e-8) + (1 - y) * np.log(1 - h + 1e-8)
    )
    return cost

def gradient_descent(X, y, lr=0.1, iterations=3000):
    m, n = X.shape
    w = np.zeros((n,1))
    b = 0
    cost_history = []

    for i in range(iterations):
        z = X @ w + b
        h = sigmoid(z)

        dw = (1/m) * (X.T @ (h - y))
        db = (1/m) * np.sum(h - y)

        w -= lr * dw
        b -= lr * db

        cost = compute_cost(X, y, w, b)
        cost_history.append(cost)

        if i % 200 == 0:
            print(f"Iter {i}, Cost: {cost}")

    return w, b, cost_history

def predict(speed, ammo, w, b, mean, std):
    x = np.array([speed, ammo])
    x_norm = (x - mean) / std
    prob = sigmoid(np.dot(x_norm, w) + b)
    # Fix for DeprecationWarning: explicitly extract scalar value
    return prob.item()


def plot_cost(costs):
    plt.plot(costs)
    plt.xlabel("Iteration")
    plt.ylabel("Cost (Loss)")
    plt.title("Cost Drop Over Training")
    plt.grid()
    plt.show()

def plot_decision_boundary(X, y, w, b):
    plt.scatter(X[:,0], X[:,1], c=y.flatten(), cmap='bwr', edgecolors='k')

    x_vals = np.linspace(min(X[:,0])-1, max(X[:,0])+1, 100)
    y_vals = -(w[0] * x_vals + b) / w[1]

    plt.plot(x_vals, y_vals, color='green', linewidth=2, label="Decision Boundary")

    plt.xlabel("Sprint Speed (normalized)")
    plt.ylabel("Ammo Clips (normalized)")
    plt.legend()
    plt.title("Decision Boundary")
    plt.show()

def main():
    X_raw, y = load_csv("zombies_data - Sheet1.csv")

    X_norm, mean, std = normalize(X_raw)

    w, b, costs = gradient_descent(X_norm, y, lr=0.1, iterations=3000)

    prob = predict(25, 1, w, b, mean, std)
    print(f"\nSurvival Probability for Runner (25 km/h, 1 Ammo Clip): {prob:.4f}")
    print("Prediction:", "SURVIVES" if prob >= 0.5 else "INFECTED")

    plot_cost(costs)
    plot_decision_boundary(X_norm, y, w, b)


if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'zombies_data - Sheet1.csv'